# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 22 2022 8:54AM,247012,10,0086000769,ISDIN Corporation,Released
1,Sep 22 2022 8:54AM,247012,10,0086000771,ISDIN Corporation,Released
2,Sep 22 2022 8:54AM,247012,10,0086000773,ISDIN Corporation,Released
3,Sep 22 2022 8:54AM,247012,10,0086000772,ISDIN Corporation,Released
4,Sep 22 2022 8:54AM,247012,10,0086000779,ISDIN Corporation,Released
5,Sep 22 2022 8:54AM,247012,10,0086000778,ISDIN Corporation,Released
6,Sep 22 2022 8:54AM,247012,10,0086000783,ISDIN Corporation,Released
7,Sep 22 2022 8:54AM,247012,10,0086000784,ISDIN Corporation,Released
8,Sep 22 2022 8:54AM,247012,10,0086000788,ISDIN Corporation,Released
9,Sep 22 2022 8:54AM,247012,10,0086000789,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,247006,Released,1
34,247007,Released,40
35,247011,Released,1
36,247012,Released,42
37,247013,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247006,NaN,NaN,1.0
247007,NaN,NaN,40.0
247011,NaN,NaN,1.0
247012,NaN,NaN,42.0
247013,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246888,0.0,1.0,0.0
246889,0.0,0.0,1.0
246890,0.0,0.0,1.0
246914,10.0,1.0,1.0
246920,10.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246888,0,1,0
246889,0,0,1
246890,0,0,1
246914,10,1,1
246920,10,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246888,0,1,0
1,246889,0,0,1
2,246890,0,0,1
3,246914,10,1,1
4,246920,10,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246888,,1,
1,246889,,,1
2,246890,,,1
3,246914,10,1,1
4,246920,10,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 22 2022 8:54AM,247012,10,ISDIN Corporation
42,Sep 22 2022 8:50AM,247013,19,"Methapharm, Inc."
48,Sep 22 2022 8:33AM,247011,19,"Granules Pharmaceuticals, Inc."
49,Sep 22 2022 8:30AM,246992,10,"Citieffe, Inc."
51,Sep 22 2022 8:27AM,247007,10,ISDIN Corporation
91,Sep 22 2022 8:22AM,247006,10,"Snap Medical Industries, LLC"
92,Sep 22 2022 8:19AM,247005,10,ISDIN Corporation
103,Sep 22 2022 8:10AM,247004,10,Emerginnova
104,Sep 22 2022 8:08AM,247003,10,Emerginnova
105,Sep 22 2022 7:42AM,247002,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 22 2022 8:54AM,247012,10,ISDIN Corporation,,,42
1,Sep 22 2022 8:50AM,247013,19,"Methapharm, Inc.",,,6
2,Sep 22 2022 8:33AM,247011,19,"Granules Pharmaceuticals, Inc.",,,1
3,Sep 22 2022 8:30AM,246992,10,"Citieffe, Inc.",,,2
4,Sep 22 2022 8:27AM,247007,10,ISDIN Corporation,,,40
5,Sep 22 2022 8:22AM,247006,10,"Snap Medical Industries, LLC",,,1
6,Sep 22 2022 8:19AM,247005,10,ISDIN Corporation,,,11
7,Sep 22 2022 8:10AM,247004,10,Emerginnova,,,1
8,Sep 22 2022 8:08AM,247003,10,Emerginnova,,,1
9,Sep 22 2022 7:42AM,247002,19,"AdvaGen Pharma, Ltd",,,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 22 2022 8:54AM,247012,10,ISDIN Corporation,42,,
1,Sep 22 2022 8:50AM,247013,19,"Methapharm, Inc.",6,,
2,Sep 22 2022 8:33AM,247011,19,"Granules Pharmaceuticals, Inc.",1,,
3,Sep 22 2022 8:30AM,246992,10,"Citieffe, Inc.",2,,
4,Sep 22 2022 8:27AM,247007,10,ISDIN Corporation,40,,
5,Sep 22 2022 8:22AM,247006,10,"Snap Medical Industries, LLC",1,,
6,Sep 22 2022 8:19AM,247005,10,ISDIN Corporation,11,,
7,Sep 22 2022 8:10AM,247004,10,Emerginnova,1,,
8,Sep 22 2022 8:08AM,247003,10,Emerginnova,1,,
9,Sep 22 2022 7:42AM,247002,19,"AdvaGen Pharma, Ltd",7,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 22 2022 8:54AM,247012,10,ISDIN Corporation,42,,
1,Sep 22 2022 8:50AM,247013,19,"Methapharm, Inc.",6,,
2,Sep 22 2022 8:33AM,247011,19,"Granules Pharmaceuticals, Inc.",1,,
3,Sep 22 2022 8:30AM,246992,10,"Citieffe, Inc.",2,,
4,Sep 22 2022 8:27AM,247007,10,ISDIN Corporation,40,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 22 2022 8:54AM,247012,10,ISDIN Corporation,42.0,NaN,NaN
1,Sep 22 2022 8:50AM,247013,19,"Methapharm, Inc.",6.0,NaN,NaN
2,Sep 22 2022 8:33AM,247011,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN
3,Sep 22 2022 8:30AM,246992,10,"Citieffe, Inc.",2.0,NaN,NaN
4,Sep 22 2022 8:27AM,247007,10,ISDIN Corporation,40.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 22 2022 8:54AM,247012,10,ISDIN Corporation,42.0,0.0,0.0
1,Sep 22 2022 8:50AM,247013,19,"Methapharm, Inc.",6.0,0.0,0.0
2,Sep 22 2022 8:33AM,247011,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0
3,Sep 22 2022 8:30AM,246992,10,"Citieffe, Inc.",2.0,0.0,0.0
4,Sep 22 2022 8:27AM,247007,10,ISDIN Corporation,40.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3951540,128.0,3.0,21.0
19,1481879,16.0,15.0,17.0
20,493962,4.0,9.0,2.0
21,1481597,5.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3951540,128.0,3.0,21.0
1,19,1481879,16.0,15.0,17.0
2,20,493962,4.0,9.0,2.0
3,21,1481597,5.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,128.0,3.0,21.0
1,19,16.0,15.0,17.0
2,20,4.0,9.0,2.0
3,21,5.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,128.0
1,19,Released,16.0
2,20,Released,4.0
3,21,Released,5.0
4,10,Executing,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,20,21
Status,,,,
Completed,21.0,17.0,2.0,0.0
Executing,3.0,15.0,9.0,1.0
Released,128.0,16.0,4.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,20,21
0,Completed,21.0,17.0,2.0,0.0
1,Executing,3.0,15.0,9.0,1.0
2,Released,128.0,16.0,4.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,20,21
0,Completed,21.0,17.0,2.0,0.0
1,Executing,3.0,15.0,9.0,1.0
2,Released,128.0,16.0,4.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()